In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

import torch.cuda as t
import cupy as cu
import numpy as np

from torch.utils.tensorboard import SummaryWriter
import tensorboard

from itertools import product

In [2]:
print("PyTorch v"+torch.__version__)
print("Torchvision v"+torchvision.__version__)
print("Tensorboard v"+tensorboard.__version__)

PyTorch v1.4.0
Torchvision v0.5.0
Tensorboard v2.1.0


In [3]:
# Check if GPU is avail
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

# Support both CPU and GPU, when avail

if torch.cuda.is_available():  
  dev = "cuda:0" 
  cu = cu
else:  
  dev = "cpu"  
  cu = np

device = torch.device(dev)

# Utility function to interchangbly support CPU and GPU when avail
def gpu(t):
    return t.to(device)

GeForce RTX 2080 Ti
True


In [4]:
gpu_cpu_test = gpu(torch.zeros(4,3))
print(gpu_cpu_test)

# Not piped to gpu
t = torch.Tensor()

# uniform number type for tensor
print(t.dtype)

# gpu or cpu?
print(t.device)

# Strided is default for how tensors are laid out in mem
print(t.layout)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')
torch.float32
cpu
torch.strided


In [5]:

'''
# NVIDIA GPU Computing Tips with code example: 
When working with #cupy and #pytorch, you might have to do a 
couple things diff to turn a #numpy / cupy array into a tensor. 
First you need to copy the numpy (cupy )array, this will 
allocate new memory for numpy array . This is to prevent  
negative strides within Ndarray which are  the number of 
locations in memory between beginnings of successive array elements. 
Next, we flip it with the Axis in the array, which  
tells which entries are reversed. 
Lastly, we can finally convert the copied array to a tensor. 
dtypes are data type enforcement and cuda:0 is piping data to the 
GPU device which has an index of 0. This provides 7-9x speed up on 
processing in most scenarios for computer vision, and 20x speed up 
for analytics/non-machine vision applications like NLP. 
The following solution performs near 100% of the 
operations on the GPU:
'''

cupy_gpu = torch.tensor(
    cu.flip(cu.copy(
        cu.array([1,2,3,4,5,6])),
            axis=0),
            dtype=torch.float16, 
            device=dev
)

print(cupy_gpu)

tensor([6., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.float16)


In [6]:
# CPU Bound Op
numpy_cpu = np.arange(15).reshape(3, 5)
print(numpy_cpu)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


In [7]:
## Start the example using Torch+Tensorboard

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [8]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
       
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        
        return t

In [9]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [10]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [11]:
### Starting out with TensorBoard (Network Graph and Images)

In [12]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

In [13]:
### Training Loop Review

In [14]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(1):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 
        
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)

tensor([7, 1, 9, 0, 1, 0, 2, 9, 6, 8, 0, 3, 4, 1, 1, 2, 8, 8, 0, 1, 4, 8, 6, 3, 5, 9, 6, 2, 9, 3, 3, 1, 1, 5, 0, 7, 2,
        1, 8, 5, 3, 8, 2, 1, 9, 1, 3, 0, 0, 8, 9, 7, 9, 9, 0, 4, 0, 8, 7, 8, 9, 0, 9, 6, 4, 0, 2, 5, 0, 5, 5, 1, 8, 0,
        7, 6, 9, 3, 3, 7, 7, 1, 0, 6, 5, 0, 5, 1, 5, 6, 7, 0, 1, 2, 6, 2, 4, 3, 2, 2])
tensor([2, 9, 8, 2, 4, 3, 1, 7, 4, 3, 1, 2, 6, 1, 6, 4, 0, 4, 7, 4, 2, 3, 8, 7, 7, 7, 4, 5, 4, 6, 4, 1, 2, 2, 8, 3, 6,
        9, 2, 0, 0, 8, 0, 1, 8, 7, 5, 9, 8, 8, 9, 5, 1, 4, 4, 2, 3, 4, 4, 8, 4, 8, 7, 5, 5, 6, 9, 1, 3, 5, 0, 2, 0, 0,
        9, 9, 5, 2, 4, 7, 6, 2, 1, 5, 6, 1, 4, 9, 2, 4, 2, 7, 1, 2, 6, 9, 8, 7, 6, 4])
tensor([9, 0, 6, 7, 8, 5, 2, 2, 2, 3, 3, 4, 0, 0, 8, 9, 6, 4, 2, 4, 7, 9, 6, 2, 3, 5, 5, 0, 3, 5, 0, 4, 5, 0, 2, 7, 1,
        0, 4, 6, 4, 3, 8, 0, 3, 4, 3, 8, 8, 8, 1, 3, 9, 8, 7, 5, 5, 7, 9, 0, 5, 8, 1, 8, 0, 1, 3, 9, 5, 5, 0, 0, 4, 5,
        2, 8, 5, 9, 9, 9, 4, 2, 7, 5, 2, 5, 3, 7, 7, 9, 9, 3, 9, 8, 5, 6, 1, 0, 6, 8])
tensor([2, 2, 7, 0, 1, 8,

tensor([1, 7, 3, 6, 6, 4, 1, 6, 1, 9, 0, 4, 2, 1, 7, 3, 9, 6, 2, 1, 4, 1, 0, 0, 2, 1, 4, 6, 4, 4, 2, 1, 8, 1, 5, 2, 5,
        7, 2, 7, 5, 0, 5, 6, 9, 4, 3, 6, 2, 4, 3, 3, 6, 7, 2, 5, 4, 5, 9, 3, 0, 1, 5, 7, 5, 8, 3, 2, 2, 1, 0, 0, 7, 7,
        2, 1, 0, 7, 1, 8, 5, 7, 5, 9, 1, 3, 7, 4, 9, 2, 2, 5, 0, 2, 1, 5, 4, 9, 2, 2])
tensor([4, 7, 9, 3, 2, 7, 4, 2, 1, 8, 1, 9, 5, 6, 2, 4, 6, 4, 7, 8, 8, 2, 9, 6, 8, 4, 7, 0, 7, 7, 6, 9, 7, 4, 1, 6, 7,
        5, 7, 0, 5, 5, 0, 7, 9, 8, 5, 5, 8, 7, 4, 9, 1, 9, 2, 0, 5, 9, 0, 8, 0, 2, 0, 6, 2, 8, 4, 2, 2, 9, 6, 8, 7, 7,
        7, 6, 6, 4, 1, 0, 1, 7, 0, 2, 7, 7, 9, 0, 8, 1, 6, 3, 7, 0, 9, 6, 6, 4, 5, 2])
tensor([8, 4, 2, 3, 1, 6, 9, 7, 0, 6, 3, 1, 3, 4, 1, 9, 9, 4, 2, 7, 5, 9, 5, 3, 5, 2, 2, 2, 1, 2, 4, 8, 4, 5, 0, 5, 5,
        6, 9, 7, 0, 1, 9, 0, 6, 8, 1, 3, 9, 9, 6, 4, 0, 2, 5, 4, 2, 5, 1, 7, 6, 7, 8, 9, 2, 3, 7, 9, 8, 4, 9, 9, 9, 8,
        7, 9, 0, 2, 0, 0, 5, 3, 5, 0, 1, 8, 7, 7, 1, 1, 4, 8, 0, 4, 9, 0, 3, 7, 3, 9])
tensor([1, 2, 6, 1, 7, 1,

        1, 2, 9, 6, 8, 8, 7, 5, 3, 2, 0, 8, 3, 0, 7, 5, 0, 4, 8, 1, 7, 5, 2, 6, 4, 9])
tensor([5, 4, 0, 9, 0, 0, 7, 3, 6, 2, 3, 8, 8, 8, 0, 0, 0, 4, 8, 5, 9, 2, 7, 6, 4, 1, 3, 6, 1, 0, 1, 1, 5, 0, 4, 5, 4,
        3, 4, 7, 2, 1, 6, 0, 1, 3, 9, 4, 9, 9, 5, 7, 1, 9, 7, 7, 0, 3, 4, 7, 7, 6, 9, 2, 2, 6, 4, 9, 4, 8, 7, 0, 8, 4,
        9, 7, 4, 9, 8, 4, 1, 1, 1, 0, 6, 6, 7, 4, 3, 5, 0, 7, 5, 2, 7, 5, 2, 2, 2, 8])
tensor([4, 1, 2, 9, 9, 5, 4, 7, 0, 9, 9, 2, 8, 4, 7, 4, 0, 2, 1, 5, 8, 5, 5, 6, 8, 5, 8, 6, 1, 8, 3, 0, 7, 5, 3, 8, 8,
        3, 9, 6, 7, 1, 0, 7, 9, 5, 1, 7, 5, 9, 8, 3, 9, 5, 1, 3, 8, 9, 5, 9, 0, 0, 1, 2, 2, 6, 8, 8, 7, 7, 1, 1, 6, 1,
        8, 1, 9, 6, 8, 2, 0, 8, 6, 9, 6, 1, 4, 3, 5, 9, 5, 3, 8, 6, 8, 2, 6, 0, 5, 0])
tensor([2, 1, 6, 4, 8, 0, 2, 4, 9, 2, 5, 9, 5, 6, 0, 0, 9, 2, 1, 1, 6, 5, 0, 1, 2, 3, 0, 5, 0, 2, 4, 9, 5, 5, 3, 8, 0,
        6, 0, 1, 6, 2, 4, 2, 2, 7, 8, 6, 2, 9, 5, 4, 3, 8, 3, 0, 3, 1, 9, 4, 3, 7, 0, 4, 2, 5, 5, 5, 6, 1, 2, 8, 3, 1,
        7, 8, 6, 2, 0, 7,

        7, 1, 1, 1, 3, 9, 4, 3, 5, 4, 9, 2, 8, 8, 5, 4, 5, 1, 4, 3, 5, 2, 9, 1, 3, 0])
tensor([7, 1, 2, 9, 4, 7, 0, 9, 1, 6, 1, 6, 1, 0, 3, 2, 6, 4, 0, 1, 0, 3, 4, 5, 5, 5, 3, 0, 0, 6, 0, 5, 1, 5, 3, 0, 0,
        1, 1, 9, 2, 9, 7, 1, 9, 6, 6, 6, 7, 3, 9, 7, 5, 3, 9, 3, 6, 2, 2, 1, 9, 1, 5, 5, 1, 9, 1, 5, 3, 1, 0, 6, 7, 4,
        1, 2, 6, 8, 3, 7, 9, 1, 1, 9, 2, 2, 9, 3, 2, 4, 0, 1, 0, 7, 5, 0, 4, 5, 3, 1])
tensor([8, 9, 1, 4, 6, 9, 1, 6, 6, 7, 2, 0, 7, 8, 4, 1, 4, 6, 2, 6, 3, 0, 1, 5, 7, 2, 9, 8, 0, 9, 0, 7, 4, 2, 0, 1, 3,
        0, 4, 0, 5, 4, 6, 7, 9, 7, 4, 2, 4, 9, 0, 1, 8, 4, 8, 1, 2, 5, 8, 8, 6, 6, 4, 0, 3, 4, 2, 9, 8, 2, 1, 3, 6, 8,
        4, 9, 1, 1, 0, 4, 2, 9, 8, 4, 3, 5, 0, 3, 6, 4, 9, 1, 9, 7, 8, 0, 5, 4, 9, 6])
tensor([4, 7, 8, 1, 3, 6, 7, 5, 4, 9, 3, 3, 2, 7, 6, 7, 1, 3, 4, 6, 1, 2, 7, 8, 1, 1, 7, 6, 3, 7, 3, 5, 9, 0, 1, 3, 8,
        2, 1, 9, 6, 1, 9, 5, 5, 6, 3, 7, 8, 1, 5, 5, 4, 3, 2, 0, 1, 2, 2, 7, 6, 5, 0, 7, 9, 2, 9, 4, 4, 3, 8, 6, 0, 4,
        1, 3, 2, 5, 9, 0,

tensor([7, 5, 1, 5, 8, 1, 4, 5, 9, 7, 6, 3, 3, 8, 6, 9, 3, 2, 4, 8, 4, 7, 8, 4, 9, 7, 3, 0, 4, 7, 0, 8, 2, 0, 7, 9, 6,
        4, 4, 4, 1, 1, 0, 4, 7, 7, 0, 7, 8, 2, 3, 6, 9, 3, 7, 6, 3, 8, 2, 9, 5, 2, 9, 4, 6, 5, 3, 4, 3, 9, 4, 6, 4, 7,
        1, 1, 3, 3, 3, 2, 2, 1, 0, 1, 4, 0, 2, 1, 4, 0, 3, 6, 0, 2, 7, 3, 2, 4, 1, 6])
tensor([8, 8, 8, 6, 8, 3, 0, 4, 0, 0, 4, 6, 2, 4, 2, 7, 8, 8, 0, 6, 5, 4, 7, 5, 8, 3, 2, 4, 1, 5, 7, 5, 5, 7, 2, 3, 2,
        3, 4, 6, 4, 4, 7, 3, 6, 6, 0, 0, 3, 6, 2, 1, 4, 9, 8, 4, 8, 2, 5, 6, 7, 3, 9, 6, 4, 5, 9, 2, 0, 0, 7, 6, 5, 7,
        8, 5, 3, 7, 5, 8, 8, 8, 4, 8, 1, 0, 3, 2, 7, 8, 2, 4, 6, 6, 1, 6, 3, 3, 5, 5])
tensor([4, 1, 5, 1, 7, 5, 8, 1, 5, 5, 6, 8, 8, 6, 6, 9, 6, 3, 0, 2, 8, 0, 9, 5, 5, 2, 7, 4, 1, 2, 5, 6, 3, 5, 9, 0, 3,
        5, 9, 4, 1, 0, 6, 3, 9, 4, 3, 1, 9, 4, 7, 3, 5, 0, 8, 9, 2, 9, 2, 0, 4, 1, 0, 0, 7, 2, 8, 3, 9, 5, 1, 0, 6, 5,
        8, 4, 3, 7, 8, 8, 0, 9, 4, 7, 0, 7, 5, 7, 5, 1, 8, 0, 3, 5, 9, 2, 4, 7, 3, 1])
tensor([5, 3, 4, 9, 0, 0,

...

tensor([1, 7, 7, 6, 2, 3, 2, 1, 8, 0, 6, 4, 7, 1, 7, 0, 4, 8, 5, 0, 4, 2, 6, 3, 6, 5, 0, 6, 3, 6, 5, 2, 3, 0, 1, 7, 6,
        3, 8, 2, 8, 9, 0, 9, 9, 5, 9, 5, 2, 8, 0, 4, 6, 2, 9, 3, 9, 6, 9, 9, 9, 1, 3, 3, 5, 5, 7, 7, 5, 8, 0, 6, 3, 3,
        2, 9, 4, 9, 2, 9, 9, 2, 7, 9, 2, 4, 0, 1, 0, 5, 8, 5, 8, 7, 5, 5, 5, 8, 2, 5])
tensor([2, 1, 2, 2, 7, 5, 7, 4, 6, 1, 4, 6, 0, 8, 3, 7, 1, 2, 9, 7, 3, 3, 4, 2, 7, 6, 9, 3, 0, 1, 7, 8, 4, 8, 5, 5, 3,
        8, 6, 7, 0, 1, 9, 6, 6, 7, 1, 5, 5, 1, 8, 5, 3, 8, 5, 9, 1, 5, 7, 0, 4, 1, 1, 2, 7, 5, 9, 0, 5, 2, 1, 2, 1, 3,
        4, 1, 2, 3, 6, 7, 9, 9, 2, 7, 8, 5, 5, 6, 4, 8, 2, 5, 9, 7, 0, 4, 0, 8, 3, 1])
tensor([4, 6, 6, 7, 1, 4, 7, 9, 0, 4, 5, 6, 4, 1, 6, 9, 4, 5, 3, 7, 1, 7, 8, 7, 6, 7, 3, 7, 3, 4, 0, 5, 4, 7, 6, 8, 8,
        8, 6, 0, 8, 8, 7, 9, 8, 7, 6, 8, 3, 9, 1, 7, 9, 0, 5, 0, 4, 2, 7, 5, 0, 2, 2, 3, 3, 4, 4, 1, 3, 1, 6, 7, 1, 7,
        3, 4, 9, 0, 8, 3, 2, 9, 0, 0, 6, 1, 8, 2, 3, 8, 8, 2, 7, 0, 4, 4, 2, 3, 5, 4])
tensor([3, 8, 8, 0, 3, 1,

tensor([1, 0, 1, 3, 7, 6, 5, 8, 3, 0, 1, 8, 4, 7, 5, 0, 8, 0, 8, 6, 9, 8, 6, 7, 1, 1, 9, 4, 6, 4, 0, 4, 2, 1, 3, 7, 1,
        2, 9, 4, 0, 3, 2, 0, 2, 8, 2, 5, 6, 3, 3, 9, 7, 2, 7, 6, 7, 3, 6, 6, 5, 6, 2, 0, 9, 0, 2, 1, 7, 2, 4, 8, 5, 6,
        1, 9, 1, 3, 2, 2, 5, 1, 9, 0, 3, 7, 7, 2, 0, 4, 0, 8, 1, 8, 1, 7, 1, 2, 0, 0])
tensor([3, 5, 3, 7, 5, 1, 0, 6, 8, 5, 3, 1, 8, 4, 7, 9, 7, 5, 4, 1, 4, 9, 3, 1, 8, 7, 9, 4, 7, 5, 3, 7, 8, 4, 5, 4, 7,
        9, 9, 8, 1, 6, 9, 9, 7, 7, 6, 5, 4, 0, 9, 3, 6, 8, 4, 7, 9, 0, 9, 5, 7, 3, 9, 6, 5, 5, 8, 3, 4, 8, 2, 5, 4, 3,
        8, 7, 8, 4, 7, 8, 0, 1, 9, 0, 7, 6, 4, 6, 5, 5, 2, 2, 2, 6, 1, 1, 5, 3, 4, 8])
tensor([3, 1, 8, 0, 8, 0, 8, 8, 2, 8, 6, 7, 1, 0, 9, 1, 4, 9, 4, 8, 5, 6, 8, 7, 0, 5, 9, 8, 4, 3, 6, 0, 4, 6, 9, 6, 9,
        9, 1, 6, 1, 4, 2, 0, 7, 9, 0, 1, 9, 0, 7, 0, 5, 8, 0, 4, 1, 8, 5, 2, 3, 7, 8, 2, 8, 0, 8, 7, 4, 5, 7, 1, 0, 9,
        2, 9, 8, 6, 9, 3, 4, 7, 2, 3, 9, 5, 7, 8, 6, 0, 3, 1, 2, 8, 3, 9, 3, 0, 2, 9])
tensor([2, 0, 1, 6, 0, 7,

tensor([1, 0, 7, 4, 5, 0, 8, 3, 5, 5, 7, 0, 8, 7, 4, 7, 2, 4, 9, 5, 1, 6, 1, 6, 0, 8, 9, 6, 0, 3, 8, 6, 9, 9, 8, 0, 7,
        8, 7, 9, 6, 0, 3, 2, 0, 9, 8, 5, 6, 0, 1, 0, 0, 1, 3, 0, 3, 9, 6, 5, 3, 8, 9, 2, 2, 3, 8, 9, 1, 4, 1, 9, 9, 9,
        1, 6, 4, 4, 9, 0, 5, 3, 5, 2, 4, 8, 9, 0, 4, 1, 9, 0, 5, 6, 9, 1, 5, 9, 3, 0])
tensor([5, 3, 6, 9, 6, 3, 9, 6, 3, 6, 7, 1, 4, 9, 8, 1, 0, 0, 6, 5, 3, 7, 5, 1, 8, 0, 7, 9, 9, 2, 8, 2, 7, 4, 1, 3, 7,
        8, 2, 0, 9, 9, 5, 3, 4, 1, 6, 8, 5, 5, 0, 5, 7, 9, 7, 4, 2, 3, 6, 1, 4, 3, 5, 4, 6, 4, 4, 6, 6, 2, 0, 1, 9, 3,
        4, 7, 4, 2, 1, 0, 2, 8, 9, 2, 3, 5, 7, 4, 4, 4, 2, 4, 8, 7, 4, 6, 6, 8, 8, 3])
tensor([9, 1, 8, 3, 6, 7, 4, 3, 0, 5, 8, 5, 1, 4, 6, 7, 7, 7, 5, 4, 2, 9, 1, 2, 7, 7, 3, 4, 1, 6, 9, 6, 5, 0, 7, 0, 3,
        1, 0, 6, 2, 7, 6, 6, 0, 1, 4, 5, 1, 4, 8, 0, 9, 5, 4, 8, 8, 4, 5, 5, 0, 8, 9, 3, 3, 9, 1, 4, 1, 7, 0, 8, 0, 2,
        3, 8, 1, 8, 1, 7, 5, 8, 8, 6, 4, 3, 2, 3, 0, 9, 0, 0, 6, 6, 2, 9, 3, 8, 9, 2])
tensor([9, 8, 2, 8, 8, 2,

tensor([3, 3, 0, 9, 5, 5, 7, 1, 2, 6, 2, 1, 7, 1, 4, 7, 0, 4, 7, 9, 0, 1, 1, 9, 4, 8, 8, 2, 4, 5, 0, 5, 3, 4, 4, 5, 0,
        5, 3, 4, 6, 0, 3, 6, 6, 2, 2, 8, 2, 0, 6, 1, 8, 3, 2, 7, 7, 3, 4, 8, 3, 6, 6, 7, 8, 8, 7, 4, 8, 9, 5, 0, 1, 9,
        1, 4, 9, 0, 1, 4, 8, 6, 1, 2, 7, 5, 6, 4, 7, 6, 3, 8, 2, 0, 1, 0, 3, 9, 2, 5])
tensor([3, 5, 5, 0, 6, 8, 1, 7, 4, 9, 3, 9, 0, 4, 4, 4, 4, 3, 7, 2, 7, 6, 8, 2, 6, 2, 9, 2, 6, 4, 3, 9, 6, 4, 7, 8, 7,
        8, 2, 5, 4, 2, 5, 2, 4, 7, 4, 4, 0, 4, 7, 0, 6, 5, 5, 6, 4, 5, 2, 5, 9, 9, 0, 1, 7, 4, 7, 5, 1, 5, 9, 8, 9, 0,
        4, 3, 5, 1, 6, 9, 7, 1, 2, 0, 0, 9, 9, 6, 1, 4, 8, 6, 1, 0, 6, 1, 3, 6, 1, 8])
tensor([2, 8, 8, 3, 3, 0, 9, 5, 6, 7, 5, 3, 3, 1, 0, 6, 5, 3, 9, 4, 7, 4, 9, 3, 7, 3, 4, 5, 9, 5, 4, 6, 6, 3, 4, 2, 9,
        5, 7, 4, 3, 0, 0, 1, 4, 1, 6, 7, 1, 5, 6, 1, 9, 3, 8, 0, 6, 9, 2, 6, 5, 0, 1, 5, 9, 3, 6, 5, 5, 2, 2, 9, 7, 0,
        7, 3, 5, 1, 3, 8, 0, 3, 8, 1, 5, 1, 7, 6, 1, 5, 1, 7, 4, 2, 9, 1, 5, 7, 6, 9])
tensor([5, 8, 7, 9, 6, 0,

tensor([5, 9, 8, 2, 6, 1, 4, 5, 9, 9, 5, 6, 2, 7, 7, 9, 0, 9, 5, 4, 8, 9, 4, 0, 7, 8, 4, 0, 8, 9, 2, 1, 0, 6, 7, 6, 2,
        6, 7, 0, 9, 1, 1, 1, 8, 5, 7, 0, 7, 2, 9, 3, 0, 0, 5, 4, 4, 5, 3, 7, 6, 0, 7, 3, 8, 3, 0, 2, 1, 6, 3, 2, 9, 2,
        0, 1, 4, 6, 3, 4, 2, 6, 8, 0, 3, 5, 7, 0, 6, 0, 5, 9, 4, 7, 4, 1, 1, 4, 7, 7])
tensor([2, 7, 3, 5, 8, 4, 3, 1, 9, 4, 1, 9, 7, 7, 2, 3, 8, 1, 1, 8, 9, 6, 4, 8, 6, 7, 5, 2, 3, 1, 1, 3, 1, 6, 6, 8, 5,
        3, 0, 9, 6, 8, 7, 1, 8, 4, 2, 4, 5, 0, 8, 2, 2, 7, 2, 8, 5, 3, 8, 8, 2, 1, 8, 6, 2, 7, 4, 1, 0, 2, 4, 2, 7, 0,
        5, 7, 0, 9, 9, 7, 9, 6, 4, 6, 1, 3, 0, 3, 0, 5, 9, 6, 0, 2, 8, 9, 0, 2, 7, 0])
tensor([6, 5, 1, 7, 5, 7, 9, 8, 4, 6, 5, 7, 5, 3, 9, 2, 6, 0, 1, 2, 6, 2, 0, 0, 1, 0, 3, 7, 7, 1, 3, 7, 4, 6, 6, 5, 2,
        7, 5, 5, 5, 2, 6, 6, 6, 0, 3, 3, 9, 1, 1, 7, 3, 1, 1, 7, 9, 3, 2, 4, 1, 7, 0, 9, 6, 9, 9, 7, 5, 8, 9, 9, 2, 5,
        1, 0, 1, 0, 3, 7, 5, 6, 9, 1, 1, 1, 4, 9, 5, 7, 4, 7, 8, 1, 1, 5, 2, 8, 2, 2])
tensor([1, 9, 1, 9, 1, 8,

In [15]:
# Training Loop with Tensorboard:

In [16]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    
    print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()

epoch 0 total_correct: 46826 loss: 346.71977780759335
epoch 1 total_correct: 51116 loss: 241.17929303646088
epoch 2 total_correct: 51904 loss: 218.1449271440506
epoch 3 total_correct: 52312 loss: 209.6566752642393
epoch 4 total_correct: 52568 loss: 200.54023799300194
epoch 5 total_correct: 52664 loss: 198.9328778833151
epoch 6 total_correct: 52945 loss: 193.04317285865545
epoch 7 total_correct: 52964 loss: 190.8975948765874
epoch 8 total_correct: 53092 loss: 187.77830977737904
epoch 9 total_correct: 52999 loss: 188.59863257408142


In [17]:
### CNN Training Hyperparamters - Neural Networks

In [18]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(5):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    #tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    #tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    #tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    
    for name, weight in network.named_parameters():
        tb.add_histogram(name, weight, epoch)
        tb.add_histogram(f'{name}.grad', weight.grad, epoch)
    
    print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()

epoch 0 total_correct: 47491 loss: 330.1289923787117
epoch 1 total_correct: 51656 loss: 225.63628886640072
epoch 2 total_correct: 52331 loss: 206.93258303403854
epoch 3 total_correct: 52756 loss: 196.37249332666397
epoch 4 total_correct: 52960 loss: 189.98838041722775


In [19]:
for name, weight in network.named_parameters():
    print(name, weight.shape)

conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([12, 6, 5, 5])
conv2.bias torch.Size([12])
fc1.weight torch.Size([120, 192])
fc1.bias torch.Size([120])
fc2.weight torch.Size([60, 120])
fc2.bias torch.Size([60])
out.weight torch.Size([10, 60])
out.bias torch.Size([10])


In [20]:
for name, weight in network.named_parameters():
    print(f'{name}.grad', weight.grad.shape)

conv1.weight.grad torch.Size([6, 1, 5, 5])
conv1.bias.grad torch.Size([6])
conv2.weight.grad torch.Size([12, 6, 5, 5])
conv2.bias.grad torch.Size([12])
fc1.weight.grad torch.Size([120, 192])
fc1.bias.grad torch.Size([120])
fc2.weight.grad torch.Size([60, 120])
fc2.bias.grad torch.Size([60])
out.weight.grad torch.Size([10, 60])
out.bias.grad torch.Size([10])


In [21]:
### Paramterized Hyperparameters

In [22]:
batch_size = 100
lr = 0.01

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

comment = f' batch_size={batch_size} lr={lr}'
tb = SummaryWriter(comment=comment)
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(2):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images, labels = batch # Get Batch
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss
        optimizer.zero_grad() # Zero Gradients
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item() * batch_size
        total_correct += get_num_correct(preds, labels)
    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    for name, param in network.named_parameters():
        tb.add_histogram(name, param, epoch)
        tb.add_histogram(f'{name}.grad', param.grad, epoch)
    
    print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)  
tb.close()

epoch 0 total_correct: 46903 loss: 34464.47133719921
epoch 1 total_correct: 51225 loss: 23569.42854076624


In [23]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]

for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
        optimizer = optim.Adam(network.parameters(), lr=lr)

        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)

        for epoch in range(1):
            total_loss = 0
            total_correct = 0
            for batch in train_loader:
                images, labels = batch # Get Batch
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels) # Calculate Loss
                optimizer.zero_grad() # Zero Gradients
                loss.backward() # Calculate Gradients
                optimizer.step() # Update Weights

                total_loss += loss.item() * batch_size
                total_correct += get_num_correct(preds, labels)

            tb.add_scalar('Loss', total_loss, epoch)
            tb.add_scalar('Number Correct', total_correct, epoch)
            tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

            for name, param in network.named_parameters():
                tb.add_histogram(name, param, epoch)
                tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)  
        tb.close()

epoch 0 total_correct: 47164 loss: 34377.579203248024
epoch 0 total_correct: 42359 loss: 46911.460906267166
epoch 0 total_correct: 32978 loss: 82537.5045478344
epoch 0 total_correct: 9665 loss: 137400.0630378723
epoch 0 total_correct: 35947 loss: 62876.280069351196
epoch 0 total_correct: 27645 loss: 91603.55013608932
epoch 0 total_correct: 8544 loss: 137580.7294845581
epoch 0 total_correct: 7532 loss: 138231.54735565186
epoch 0 total_correct: 17234 loss: 125542.97089576721
epoch 0 total_correct: 10280 loss: 137826.29251480103
epoch 0 total_correct: 6695 loss: 138222.18418121338
epoch 0 total_correct: 6000 loss: 138381.67190551758


In [24]:
parameters = dict(
    lr = [.01, .001]
    ,batch_size = [10, 100, 1000]
    ,shuffle = [True, False]
)

In [25]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

In [ ]:
for lr, batch_size, shuffle in product(*param_values): 
    print (lr, batch_size, shuffle)

In [ ]:
for lr, batch_size, shuffle in product(*param_values): 
    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'
    network = Network()
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
    optimizer = optim.Adam(network.parameters(), lr=lr)
    images, labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)
    tb = SummaryWriter(comment=comment)
    tb.add_image('images', grid)
    tb.add_graph(network, images)
    for epoch in range(25):
        total_loss = 0
        total_correct = 0
        for batch in train_loader:
            images, labels = batch # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item() * batch_size
            total_correct += get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar('Number Correct', total_correct, epoch)
        tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

        for name, param in network.named_parameters():
            tb.add_histogram(name, param, epoch)
            tb.add_histogram(f'{name}.grad', param.grad, epoch)

        print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)  
    tb.close()

epoch 0 total_correct: 45615 loss: 38141.87968805432
epoch 1 total_correct: 48174 loss: 31882.11797460448
epoch 2 total_correct: 48613 loss: 31699.818434980698
epoch 3 total_correct: 48796 loss: 31066.618630876765
epoch 4 total_correct: 48951 loss: 30445.33771365881
epoch 5 total_correct: 48765 loss: 31659.063687484595
epoch 6 total_correct: 48864 loss: 31293.54056049924
epoch 7 total_correct: 48997 loss: 31235.23635968304
epoch 8 total_correct: 47952 loss: 33445.68150140345
epoch 9 total_correct: 46682 loss: 36012.40750961006
epoch 10 total_correct: 47176 loss: 34848.395975106396
epoch 11 total_correct: 47105 loss: 34880.371011584066
epoch 12 total_correct: 47389 loss: 34280.30389838852
epoch 13 total_correct: 47402 loss: 34691.20165117085
epoch 14 total_correct: 47378 loss: 35072.040968546644
epoch 15 total_correct: 47365 loss: 33875.645976774395
epoch 16 total_correct: 47204 loss: 35074.63471646421
epoch 17 total_correct: 47384 loss: 33904.99438134022
epoch 18 total_correct: 47564 l

epoch 2 total_correct: 52865 loss: 19270.715426348615
epoch 3 total_correct: 53344 loss: 17799.080511750653
epoch 4 total_correct: 53778 loss: 16781.522462792927
epoch 5 total_correct: 53972 loss: 16045.735432240326
epoch 6 total_correct: 54226 loss: 15309.890713571804
epoch 7 total_correct: 54428 loss: 14790.602849135903
epoch 8 total_correct: 54603 loss: 14310.120066608652
epoch 9 total_correct: 54751 loss: 13943.074009879347
epoch 10 total_correct: 54934 loss: 13444.477807877993
epoch 11 total_correct: 55009 loss: 13116.742840233273
epoch 12 total_correct: 55112 loss: 12796.360957755678
epoch 13 total_correct: 55186 loss: 12468.051352632374
epoch 14 total_correct: 55356 loss: 12153.87839049643
epoch 15 total_correct: 55496 loss: 11815.242543018703
epoch 16 total_correct: 55577 loss: 11667.007423964864
epoch 17 total_correct: 55655 loss: 11319.77843435346
epoch 18 total_correct: 55759 loss: 11086.281989868412
epoch 19 total_correct: 55766 loss: 10926.133343640451
epoch 20 total_corre